# In this Notebook, I am going to test a Deep Reinforcement Learning Strategy

In [ ]:
from preprocessing import preprocessors
from model import models

## This file contains functions to build the actual models

# common libraries
import pandas as pd 
import numpy as np
import time
import gym

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

# stable_baselines RL stuff
from stable_baselines import GAIL, SAC
from stable_baselines import ACER
from stable_baselines import PPO2
from stable_baselines import A2C
from stable_baselines import DDPG
from stable_baselines import TD3

from stable_baselines.ddpg.policies import DDPGPolicy
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines.common.vec_env import DummyVecEnv


# Custom Functions
from preprocessing.preprocessors import *
from config import config

# Custom Environments
from env.EnvMultipleAssets_train import AssetEnvTrain
from env.EnvMultipleAssets_validation import AssetEnvValidation
from env.EnvMultipleAssets_trade import AssetEnvTrade

In [ ]:
data = preprocessors.preprocess_data()

In [ ]:
data = preprocessors.add_turbulence(data)

In [ ]:
# train through 2015, then validate, then test from 2018 on
unique_trade_date = data[(data['date'] > "2009-01-01")&(data['date'] <= "2020-12-01")].date.unique()

# rebalance_window is the number of months to retrain the model
# validation_window is the number of months to validate the models 
# and select for trading
rebalance_window = 63
validation_window = 63

In [ ]:
models.run_ensemble_strategy(df = data, 
                             unique_trade_date = unique_trade_date, 
                             rebalance_window = rebalance_window, 
                             validation_window = validation_window)

# Put all Trade Results Together

In [ ]:
import os
file_list = os.listdir('results')

In [ ]:
file_list = [file for file in file_list if ('account_value_trade' in file) & ('csv' in file)]

In [ ]:
file_list.sort()
test = [file.split('.') for file in file_list]
test = [file[0].split('_') for file in test]
test = [file[4] for file in test]
test = [int(file) for file in test]

In [ ]:
files = pd.DataFrame({'file_name':file_list, 'file_number':test})
files = files.sort_values('file_number')

In [ ]:
results_df = pd.DataFrame()
for file in files['file_name']:
    if 'account_value_trade' in str(file):
        if 'csv' in str(file):
            temp_df = pd.read_csv('results/' + file)
            results_df = results_df.append(temp_df)

In [ ]:
results_df.drop(results_df.columns[0], axis=1, inplace=True)
results_df.columns = ['Portfolio_Value']

In [ ]:
results_df = results_df.reset_index(drop=True)
trade_dates = data[(data['date'] > '2009-12-11') & (data['date'] <= '2020-11-26')]['date'].unique()

In [ ]:
plt.clf()
sns.lineplot(x=trade_dates, y=results_df[results_df.columns[0]])
plt.savefig('total_results_plot.png')